In [1]:
import tensorflow as tf
print(tf.__version__)

2.13.0


In [2]:
import numpy as np
import matplotlib.pyplot as plt
def plot_series(time, series, format="-", start=0, end=None):
    plt.plot(time[start:end], series[start:end], format)
    plt.xlabel("Time")
    plt.ylabel("Value")
    plt.grid(True)

In [3]:
def windowed_dataset(series, batch_size, n_past=24, n_future=24, shift=1):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(size=n_past + n_future, shift=shift, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(n_past + n_future))
    ds = ds.map(lambda w: (w[:n_past], w[n_past:]))
    return ds.batch(batch_size).prefetch(1)

In [4]:
def model_forecast(model, series, window_size):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size))
    ds = ds.batch(32).prefetch(1)
    forecast = model.predict(ds)
    return forecast

In [5]:
def normalize_series(data, min, max):
    data = data - min
    data = data / max
    return data

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional


Sunspot

In [7]:
import tensorflow as tf
print(tf.__version__)

2.13.0


In [8]:
from sklearn.preprocessing import MinMaxScaler


In [9]:
!wget --no-check-certificate \
    https://storage.googleapis.com/download.tensorflow.org/data/certificate/household_power.zip \
    -O /tmp/household_power.zip

--2023-10-12 15:00:10--  https://storage.googleapis.com/download.tensorflow.org/data/certificate/household_power.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.2.207, 2607:f8b0:4023:c06::cf, 2607:f8b0:4023:c0d::cf
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.2.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 859850 (840K) [application/zip]
Saving to: ‘/tmp/household_power.zip’

/tmp/household_powe 100%[===================>] 839.70K  --.-KB/s    in 0.007s  

2023-10-12 15:00:10 (114 MB/s) - ‘/tmp/household_power.zip’ saved [859850/859850]



In [10]:
# !unzip /tmp/household_power.zip -d /tmp

In [11]:
import pandas as pd
df = pd.read_csv('/tmp/household_power_consumption.csv', sep=',',
                     infer_datetime_format=True, index_col='datetime', header=0)

N_FEATURES = len(df.columns) # DO NOT CHANGE THIS

data = df.values
data = normalize_series(data, data.min(axis=0), data.max(axis=0))

SPLIT_TIME = int(len(data) * 0.5) # DO NOT CHANGE THIS
x_train = data[:SPLIT_TIME]
x_valid = data[SPLIT_TIME:]

tf.keras.backend.clear_session()
tf.random.set_seed(42)
BATCH_SIZE = 32  # ADVISED NOT TO CHANGE THIS


N_PAST = 24  # DO NOT CHANGE THIS

N_FUTURE = 24  # DO NOT CHANGE THIS

SHIFT = 1  # DO NOT CHANGE THIS

train_set = windowed_dataset(series=x_train, batch_size=BATCH_SIZE,
                                 n_past=N_PAST, n_future=N_FUTURE,
                                 shift=SHIFT)
valid_set = windowed_dataset(series=x_valid, batch_size=BATCH_SIZE,
                                 n_past=N_PAST, n_future=N_FUTURE,
                                 shift=SHIFT)

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, LSTM, Dense, Bidirectional
from tensorflow.keras.backend import clear_session
from tensorflow.keras.callbacks import ModelCheckpoint

Conv1D

In [18]:
checkpoint = ModelCheckpoint(filepath='/home/mymodel.h5', monitor='mae', verbose=0, save_best_only=True)
model = Sequential([
      Bidirectional(LSTM(16, return_sequences=True),input_shape = (N_PAST, N_FEATURES)),
      tf.keras.layers.Dense(N_FEATURES)
])
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])
history = model.fit(
    train_set,
    epochs=50,
    validation_data=valid_set)

Epoch 1/50
1349/1349 [==============================] - 30s 20ms/step - loss: 0.0192 - mae: 0.0757 - val_loss: 0.0171 - val_mae: 0.0683
Epoch 2/50
1349/1349 [==============================] - 36s 27ms/step - loss: 0.0170 - mae: 0.0673 - val_loss: 0.0161 - val_mae: 0.0644
Epoch 3/50
1349/1349 [==============================] - 36s 27ms/step - loss: 0.0162 - mae: 0.0648 - val_loss: 0.0152 - val_mae: 0.0620
Epoch 4/50
1349/1349 [==============================] - 27s 20ms/step - loss: 0.0153 - mae: 0.0633 - val_loss: 0.0142 - val_mae: 0.0605
Epoch 5/50
1349/1349 [==============================] - 28s 21ms/step - loss: 0.0144 - mae: 0.0611 - val_loss: 0.0137 - val_mae: 0.0590
Epoch 6/50
1349/1349 [==============================] - 26s 20ms/step - loss: 0.0140 - mae: 0.0597 - val_loss: 0.0134 - val_mae: 0.0581
Epoch 7/50
1349/1349 [==============================] - 26s 20ms/step - loss: 0.0137 - mae: 0.0587 - val_loss: 0.0132 - val_mae: 0.0575
Epoch 8/50
1349/1349 [==========================

In [16]:
model.save("/home/mymodel.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
